# Setup

In [1]:
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.metrics import f1_score, cohen_kappa_score, accuracy_score, confusion_matrix

# Import Data

In [2]:
impt_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")

In [3]:
import datetime

def d2(time_date):
    return(datetime.datetime.strptime(time_date, '%Y-%m-%d %H:%M:%S'))

def d1(time_date):
    return(datetime.datetime.strptime(time_date, '%Y-%m-%d %H:%M:%S'))

In [4]:
differences = [d2(time_date_1) - d1(time_date_2) for time_date_1, time_date_2 in zip(impt_data['listing_date'], impt_data['issue_date'])]
time_diff_col = pd.Series([(difference.days + difference.seconds/86400)/365.2425 for difference in differences]) 

In [5]:
differences = [d2(time_date_1) - d1(time_date_2) for time_date_1, time_date_2 in zip(test_data['listing_date'], test_data['issue_date'])]
time_diff_col_test = pd.Series([(difference.days + difference.seconds/86400)/365.2425 for difference in differences]) 

In [6]:
impt_data["year_diff"] = time_diff_col
test_data["year_diff"] = time_diff_col_test

# Pre Processing

In [7]:
impt_data.fillna(0, inplace = True)
test_data.fillna(0, inplace = True)

In [8]:
bc_labels_pd = impt_data.pop("breed_category")
pc_labels_pd = impt_data.pop("pet_category")

In [9]:
include_columns = ["condition", "color_type", "length(m)", "height(cm)", "X1", "X2", "year_diff"]
data_pd = impt_data[include_columns]
data_test_pd = test_data[include_columns]

In [10]:
# turn X into dict
data_dict = data_pd.to_dict(orient='records') # turn each row as key-value pairs
data_test_dict = data_test_pd.to_dict(orient='records') # turn each row as key-value pairs

In [11]:
# DictVectorizer
from sklearn.feature_extraction import DictVectorizer
# instantiate a Dictvectorizer object for X
dv_X = DictVectorizer(sparse=False) 
# sparse = False makes the output is not a sparse matrix

# apply dv_X on X_dict
data = dv_X.fit_transform(data_dict)
X_test = dv_X.transform(data_test_dict)
# show X_encoded
X_test

array([[ 0.        ,  7.        ,  0.        , ..., 42.73      ,
         0.87      , 12.0595202 ],
       [ 0.        ,  1.        ,  0.        , ...,  6.71      ,
         0.06      ,  0.4783808 ],
       [ 0.        ,  7.        ,  0.        , ..., 41.21      ,
         0.24      ,  5.47499835],
       ...,
       [ 0.        ,  7.        ,  0.        , ..., 37.19      ,
         0.98      ,  1.07797293],
       [ 0.        ,  2.        ,  0.        , ..., 23.83      ,
         0.79      ,  1.06165957],
       [ 0.        ,  1.        ,  0.        , ..., 24.51      ,
         0.64      ,  1.07517418]])

In [12]:
# # Categorical boolean mask
# categorical_feature_mask = data_pd.dtypes==object

# # filter categorical columns using mask and turn it into a list
# categorical_cols = data_pd.columns[categorical_feature_mask].tolist()

In [13]:
# # instantiate labelencoder object
# le = LabelEncoder()

# # apply le on categorical feature columns
# data_pd[categorical_cols] = data_pd[categorical_cols].apply(lambda col: le.fit_transform(col))
# data_pd[categorical_cols].head(10)

# data_test_pd[categorical_cols] = data_test_pd[categorical_cols].apply(lambda col: le.fit_transform(col))
# data_test_pd[categorical_cols].head(10)

In [14]:
# data = data_pd.to_numpy()
# X_test = data_test_pd.to_numpy()
bc_labels = bc_labels_pd.to_numpy()
pc_labels = pc_labels_pd.to_numpy()
labels = np.stack((bc_labels, pc_labels), axis = -1)
print(data.shape)
print(X_test.shape)
print(pc_labels.shape)

(18834, 62)
(8072, 62)
(18834,)


# Accuracy Score

In [15]:
def accuracy(Y_hat, Y):
    accuracy = accuracy_score(np.argmax(Y_hat.cpu().detach().numpy(), 1), np.argmax(Y.cpu().detach().numpy(), 1))
    f1_scr = f1_score(np.argmax(Y_hat.cpu().detach().numpy(), 1), np.argmax(Y.cpu().detach().numpy(), 1), average='weighted')
    kappa = cohen_kappa_score(np.argmax(Y_hat.cpu().detach().numpy(), 1), np.argmax(Y.cpu().detach().numpy(), 1))
    return(accuracy, f1_scr, kappa)

In [16]:
def accuracy_ml(Y_hat, Y):
    accuracy = accuracy_score(Y_hat, Y)
    f1_scr = f1_score(Y_hat,Y, average='weighted')
    kappa = cohen_kappa_score(Y_hat,Y)
    return(accuracy, f1_scr, kappa)

# Train Test Split

In [158]:
X_train, X_val, Y_train, Y_val = train_test_split(data, pc_labels, random_state = 0, test_size = 0.3)

In [232]:
# # META CODE
# from sklearn.multiclass import OneVsRestClassifier
# from xgboost import XGBClassifier
# from sklearn.preprocessing import MultiLabelBinarizer

# # clf = OneVsRestClassifier(XGBClassifier(n_jobs=-1, max_depth=9))

# # # You may need to use MultiLabelBinarizer to encode your variables from arrays [[x, y, z]] to a multilabel 
# # # format before training.
# # mlb = MultiLabelBinarizer()
# # # Y_OH_train = mlb.fit_transform(Y_train)

# clf = XGBClassifier(eta = 0.2, n_jobs=-1, max_depth=7, reg_lambda = 0.6, tree_method = "auto", alpha = 0.5,
#                    gamma = 2, sampling_method = "gradient_based")
# #                     colsample_bytree, colsample_bylevel, colsample_bynode)
# clf.fit(X_train, Y_train)

# META CODE
from sklearn.multiclass import OneVsRestClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import MultiLabelBinarizer

# clf = OneVsRestClassifier(XGBClassifier(n_jobs=-1, max_depth=9))

# # You may need to use MultiLabelBinarizer to encode your variables from arrays [[x, y, z]] to a multilabel 
# # format before training.
# mlb = MultiLabelBinarizer()
# # Y_OH_train = mlb.fit_transform(Y_train)

clf = XGBClassifier(eta = 0.2, n_jobs=-1, max_depth=8, reg_lambda = 0.6, tree_method = "auto", alpha = 0.4,
                   gamma = 3, sampling_method = "uniform", n_estimators = 50)
#                     ,colsample_bytree = 0.9, colsample_bylevel = 0.9, colsample_bynode = 0.9)
clf.fit(X_train, Y_train)

XGBClassifier(alpha=0.4, base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eta=0.2, gamma=3,
              gpu_id=-1, importance_type='gain', interaction_constraints='',
              learning_rate=0.200000003, max_delta_step=0, max_depth=8,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=50, n_jobs=-1, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0.400000006,
              reg_lambda=0.6, sampling_method='uniform', scale_pos_weight=None,
              subsample=1, tree_method='auto', validate_parameters=1,
              verbosity=None)

In [233]:
print(accuracy_ml(clf.predict(X_train), Y_train))
print(accuracy_ml(clf.predict(X_val), Y_val))

(0.8996434802397026, 0.9014853731376652, 0.8082324215122889)
(0.8918775437975579, 0.8934986229876767, 0.7936479448829157)


In [ ]:
enc = OneHotEncoder()
Y_OH_train = enc.fit_transform(np.expand_dims(Y_train, 1)).toarray()
Y_OH_val = enc.transform(np.expand_dims(Y_val, 1)).toarray()

In [ ]:
pc_labels

In [ ]:
Y_OH_train[:40]

In [ ]:
X_train.shape

In [ ]:
X_train, Y_OH_train, X_val, Y_OH_val, X_test = map(torch.tensor, (X_train, Y_OH_train, X_val, Y_OH_val, X_test))

In [ ]:
X_train, X_val, X_test = X_train.float(), X_val.float(), X_test.float()

# Cuda Support

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = 'cpu'

X_train = X_train.to(device)
X_val = X_val.to(device)
X_test  = X_test.to(device)
Y_OH_train = Y_OH_train.to(device)
Y_OH_val = Y_OH_val.to(device)

In [ ]:
Y_OH_train.shape

# FF Network

In [ ]:
class FF_Network(nn.Module):
    def __init__(self):
        super().__init__()
        torch.manual_seed(0)
        self.net = nn.Sequential(
            nn.Linear(62, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(),
            
            nn.Linear(256,128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(),
            
            nn.Linear(128,512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            
            nn.Linear(512, 64),
            nn.BatchNorm1d(64),
            nn.Tanh(),
            
            nn.Linear(64, 16),
            nn.BatchNorm1d(16),
            nn.Tanh(),
            
            nn.Linear(16, 4)
        )
           
    def forward(self, X):
        return(self.net(X))

In [ ]:
ff_n = FF_Network()
ff_n = ff_n.to(device)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
def train(X, Y, model, loss_fn = loss_fn, lr = 0.02, epochs = 2500, batch_size = 512):
    loss_b = []
    acc_b = []
    acc = 0
    optm = optim.Adam(ff_n.parameters(), lr = lr)
    Y_temp = Y.to('cpu')
    Y = np.argmax(Y_temp, 1)
    Y = Y.to(device)
    for i in tqdm_notebook(range(epochs)):
        for i in range(X.shape[0] // batch_size):
            local_X, local_Y, local_y_temp = X[i*batch_size:(i+1)*batch_size,], Y[i*batch_size:(i+1)*batch_size,], Y_temp[i*batch_size:(i+1)*batch_size,]
            Y_hat = model.forward(local_X)
            loss = loss_fn(Y_hat, local_Y)
            acc, _, _ = accuracy(Y_hat, local_y_temp)
            loss_b.append(loss.item())
            acc_b.append(acc)
            loss.backward()
            optm.step()
            optm.zero_grad()
    return(loss_b, acc_b, acc)

In [ ]:
%%time
loss_b, acc_b, acc_score = train(X_train, Y_OH_train, ff_n, loss_fn, lr = 0.001, epochs = 2500, batch_size = 8192)

In [ ]:
fig = plt.figure(figsize = (16, 8))
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)
ax1.plot(loss_b)
ax1.set_xlabel('time progress')
ax1.set_ylabel('loss')
ax2.plot(acc_b)
ax2.set_xlabel('time_progress')
ax2.set_ylabel('acc')

In [ ]:
acc_b

In [ ]:
# ff_n.eval()
print("Train set accuracy, f1 score, kappa: ", accuracy(ff_n.forward(X_train), Y_OH_train), "\nValidation set accuracy, f1 score: ", accuracy(ff_n.forward(X_val), Y_OH_val))#, "\nTest set accuracy, f1 score: ", accuracy(ff_n.forward(X_test), Y_OH_test))

# Visual Evaluation

In [ ]:
cm = confusion_matrix(np.argmax(Y_OH_val.to('cpu').detach().numpy(), 1), np.argmax(ff_n(X_val).to('cpu').detach().numpy(), 1))
print(cm)
plt.imshow(cm, cmap='binary')

In [ ]:
cm = confusion_matrix(np.argmax(Y_OH_train.to('cpu').detach().numpy(), 1), np.argmax(ff_n(X_train).to('cpu').detach().numpy(), 1))
print(cm)
plt.imshow(cm, cmap='binary')

# Saving Model

In [ ]:
torch.save(ff_n.state_dict(), './weights')

In [ ]:
model = FF_Network()
model.load_state_dict(torch.load('./weights'))
model.eval()
model.to(device)

In [ ]:
print("Train set Accuracy, F1 score, Cohen's Kappa: ", accuracy(model.forward(X_train), Y_OH_train), "\nValidation set Accuracy, F1 score, Cohens Kappa: ", accuracy(model.forward(X_val), Y_OH_val))

In [157]:
import csv
Y_test_pred = clf.predict(X_test)
# Y_test_pred = model.forward(X_test)

with open('submission.csv', 'w', newline='') as file:
    with open('test.csv', 'r') as inp:
        writer = csv.writer(file)
        reader = csv.reader(inp)
        heading = next(reader)
        heading.append("class")
        writer.writerow(['class'])
        for i, row in enumerate(reader):
            writer.writerow([Y_test_pred[i]])
#             writer.writerow(enc.inverse_transform(Y_test_pred.to('cpu').detach().numpy())[i])

In [ ]:
file.close()
inp.close()

In [ ]:
import os

name = 'answer.csv'
bakname = name + '.bak'
os.rename(name, bakname)

with open(bakname, 'rb') as csv_in, open(name, 'wb') as csv_out:
    writer = csv.writer(csv_out)
    for i, row in enumerate(csv.reader(csv_in)):
        row[2] = enc.inverse_transform(Y_test_pred.to('cpu').detach().numpy())[i]
        writer.writerow(row)